In [134]:
import pandas as pd
import os

# Set paths to csv files
school_csv = os.path.join("Resources", "schools_complete.csv")
student_csv = os.path.join("Resources", "students_complete.csv")

# Declare datafram variables
school_data = pd.read_csv(school_csv)
student_data = pd.read_csv(student_csv)

# Create new pass/fail columns.  1 if true, 0 if false 
student_data.loc[student_data["math_score"] >= 70,'math_pass_calc'] = 1
student_data.loc[student_data["math_score"] < 70,'math_pass_calc'] = 0
student_data.loc[student_data["reading_score"] >= 70,'read_pass_calc' ]= 1
student_data.loc[student_data["reading_score"] < 70,'read_pass_calc' ]= 0
student_data.loc[(student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70),'total_pass_calc' ]= 1
student_data.loc[(student_data["math_score"] < 70) | (student_data["reading_score"] < 70),'total_pass_calc' ]= 0

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")

# Create new column to show bin labels
bins = [0, 1000, 2000, 5000]
group_labels = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]
school_data_complete["School Size"] = pd.cut(school_data_complete["size"], bins, labels=group_labels)

In [109]:
# Compile values for different fields
count_schools = len(pd.unique(school_data_complete["school_name"]))
count_students = "{:,}".format(pd.unique(school_data_complete["size"]).sum())
count_budget = "${:,.2f}".format(pd.unique(school_data_complete["budget"]).sum())
avg_math = school_data_complete["math_score"].mean()
avg_read = school_data_complete["reading_score"].mean()
perc_math_pass = (school_data_complete["math_pass_calc"].sum()/len(school_data_complete))*100
perc_read_pass = (school_data_complete["read_pass_calc"].sum()/len(school_data_complete))*100
perc_total_pass = (school_data_complete["total_pass_calc"].sum()/len(school_data_complete))*100

# Insert values into fields
District_Summary = [
    {
        "Total Schools": count_schools
        ,"Total Students": count_students
        ,"Total Budget": count_budget
        ,"Average Math Score": avg_math
        ,"Average Reading Score":avg_read
        ,"% Passing Math":perc_math_pass
        ,"% Passing Reading":perc_read_pass
        ,"% Overall Passing":perc_total_pass
    }
]

# Display data
df = pd.DataFrame(data=District_Summary)
df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [129]:
# Get static data from school data
data_min = school_data.groupby("school_name").min()
data_min_slim = data_min[['type','size','budget']]

# Get average data from school data complete dataframe
data_avg = school_data_complete.groupby("school_name").mean()
data_avg_slim = data_avg[["math_score","reading_score","math_pass_calc","read_pass_calc","total_pass_calc"]]

# Merge static data with averaged data
school_summary = pd.merge(data_min_slim,data_avg_slim, how="left", on="school_name")

# Create new field to look at budget per student
school_summary.loc[:,'student budget'] = school_summary["budget"]/school_summary["size"]

# Format fields
school_summary['size'] = school_summary['size'].map('{:,}'.format)
school_summary['budget'] = school_summary['budget'].map('${:,.2f}'.format)
school_summary['math_pass_calc'] = school_summary['math_pass_calc']*100
school_summary['read_pass_calc'] = school_summary['read_pass_calc']*100
school_summary['total_pass_calc'] = school_summary['total_pass_calc']*100

# Renaming fields
school_summary_final = school_summary.rename(columns={
    "type":"School Type"
    ,"size":"Total Students"
    ,"budget":"Total School Budget"
    ,"student budget":"Per Student Budget"
    ,"math_score":"Average Math Score"
    ,"reading_score":"Average Reading Score"
    ,"math_pass_calc":"% Passing Math"
    ,"read_pass_calc":"% Passing Reading"
    ,"total_pass_calc":"% Overall Passing"
})[["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]

# Format Per Student Budget field after renaming
school_summary_final['Per Student Budget'] = school_summary_final['Per Student Budget'].map('${:,.2f}'.format)

# Display Data
school_summary_final

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [111]:
# Ordering data by % overall passing
school_summary_passing_order = school_summary_final.sort_values(by=['% Overall Passing'], ascending=False)

# Display top 5
school_summary_passing_order.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [112]:
# Reverse ordering data by % overall pass
school_summary_failing_order = school_summary_final.sort_values(by=['% Overall Passing'])

# Dispaly top 5
school_summary_failing_order.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [113]:
# Enter in math data into fields if grade equals column name
school_data_complete.loc[school_data_complete['grade'] == "9th","9th"] = school_data_complete['math_score']
school_data_complete.loc[school_data_complete['grade'] == "10th","10th"] = school_data_complete['math_score']
school_data_complete.loc[school_data_complete['grade'] == "11th","11th"] = school_data_complete['math_score']
school_data_complete.loc[school_data_complete['grade'] == "12th","12th"] = school_data_complete['math_score']

# Aggregate Average of different fields by school name
data_avg_math = school_data_complete.groupby("school_name").mean()
data_avg_math_slim = data_avg_math[["9th","10th","11th","12th"]]

# Display data
data_avg_math_slim

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [114]:
# Enter in reading data into fields if grade equals column name
school_data_complete.loc[school_data_complete['grade'] == "9th","9th"] = school_data_complete['reading_score']
school_data_complete.loc[school_data_complete['grade'] == "10th","10th"] = school_data_complete['reading_score']
school_data_complete.loc[school_data_complete['grade'] == "11th","11th"] = school_data_complete['reading_score']
school_data_complete.loc[school_data_complete['grade'] == "12th","12th"] = school_data_complete['reading_score']

# Aggregate Average of different fields by school name
data_avg_read = school_data_complete.groupby("school_name").mean()
data_avg_read_slim = data_avg_read[["9th","10th","11th","12th"]]

# Display data
data_avg_read_slim

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [143]:
# Set in bins per student spending ranges
bins = [0, 585, 630, 645,680]
group_labels = ["<$585","$585-630","$630-645","$645-680"]
school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["student budget"], bins, labels=group_labels)

# Average data by per student spending
student_spend = school_summary.groupby("Spending Ranges (Per Student)").mean()

# Format fields
student_spend.loc[:,'Average Math Score'] = student_spend['math_score'].map('{:.2f}'.format)
student_spend.loc[:,'Average Reading Score'] = student_spend['reading_score'].map('{:.2f}'.format)
student_spend.loc[:,'% Passing Math'] = student_spend['math_pass_calc'].map('{:.2f}'.format)
student_spend.loc[:,'% Passing Reading'] = student_spend['read_pass_calc'].map('{:.2f}'.format)
student_spend.loc[:,'% Overall Passing'] = student_spend['total_pass_calc'].map('{:.2f}'.format)

# Get needed columns and sort by school size category
student_spend_slim = student_spend[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]

# Show data
student_spend_slim

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


In [140]:
# Average data by school size
school_size_agg = school_data_complete.groupby("School Size").mean()

# Get needed columns and sort by school size category
school_size_agg_slim = school_size_agg[["math_score","reading_score","math_pass_calc","read_pass_calc","total_pass_calc"]].sort_values(by=['School Size'])

# Recalculate numbers so they show as percent numbers
school_size_agg_slim['math_pass_calc'] = school_size_agg_slim['math_pass_calc']*100
school_size_agg_slim['read_pass_calc'] = school_size_agg_slim['read_pass_calc']*100
school_size_agg_slim['total_pass_calc'] = school_size_agg_slim['total_pass_calc']*100

# Rename fields
school_size_agg_slim = school_size_agg_slim.rename(columns={
    "math_score":"Average Math Score"
    ,"reading_score":"Average Reading Score"
    ,"math_pass_calc":"% Passing Math"
    ,"read_pass_calc":"% Passing Reading"
    ,"total_pass_calc":"% Overall Passing"
})

# Show data
school_size_agg_slim

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


In [142]:
# Average data by school type
school_type_agg = school_data_complete.groupby("type").mean()

# Get needed columns and sort by school type
school_type_agg_slim = school_type_agg[["math_score","reading_score","math_pass_calc","read_pass_calc","total_pass_calc"]].sort_values(by=['type'])

# Recalculate numbers so they show as percent numbers
school_type_agg_slim['math_pass_calc'] = school_type_agg_slim['math_pass_calc']*100
school_type_agg_slim['read_pass_calc'] = school_type_agg_slim['read_pass_calc']*100
school_type_agg_slim['total_pass_calc'] = school_type_agg_slim['total_pass_calc']*100

# Rename fields
school_type_agg_slim = school_type_agg_slim.rename(columns={
    "math_score":"Average Math Score"
    ,"reading_score":"Average Reading Score"
    ,"math_pass_calc":"% Passing Math"
    ,"read_pass_calc":"% Passing Reading"
    ,"total_pass_calc":"% Overall Passing"
})

# Show data
school_type_agg_slim

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
